In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.datasets import load_digits
import seaborn as sns

digits = load_digits()
print("Image Data Shape", digits.data.shape)
print("Label Data Shape", digits.target.shape)

plt.figure()

Image Data Shape (1797, 64)
Label Data Shape (1797,)


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>